In [12]:
import json
from pathlib import Path

data_path = Path("/kaggle/input/advocacy-cases")
json_file = data_path / "advocacy_cases.json"  

with open(json_file, "r") as f:
    cases_raw = json.load(f)

len(cases_raw), cases_raw[0].keys()


(9,
 dict_keys(['case_id', 'patient_id', 'advocate_id', 'opened_at', 'closed_at', 'status', 'issue_type', 'priority', 'reassigned_flag', 'reopen_count', 'interactions']))

In [13]:
import pandas as pd

cases_df = pd.json_normalize(
    cases_raw,
    sep="_",  
    meta=[
        "case_id",
        "patient_id",
        "advocate_id",
        "opened_at",
        "closed_at",
        "status",
        "issue_type",
        "priority",
        "reassigned_flag",
        "reopen_count"
    ]
)

# Drop interactions column to keep only case-level fields
if "interactions" in cases_df.columns:
    cases_df = cases_df.drop(columns=["interactions"])

cases_df.head()


,case_id,patient_id,advocate_id,opened_at,closed_at,status,issue_type,priority,reassigned_flag,reopen_count
0,C-1001,P-0501,A-001,2025-11-01T09:15:00,2025-11-05T14:40:00,closed,billing,high,0,0
1,C-1002,P-0502,A-002,2025-10-01T09:15:00,2025-10-05T14:40:00,closed,billing,high,0,0
2,C-1003,P-0503,A-003,2025-10-02T09:15:00,2025-10-06T14:40:00,closed,billing,high,0,0
3,C-1004,P-0504,A-004,2025-10-01T09:15:00,2025-10-05T14:40:00,closed,billing,high,0,0
4,C-1005,P-0505,A-005,2025-10-07T09:15:00,2025-10-09T14:40:00,closed,billing,medium,0,0


In [14]:
interactions_df = pd.json_normalize(
    cases_raw,
    record_path="interactions",
    meta=[
        "case_id",
        "patient_id",
        "advocate_id",
        "opened_at",
        "closed_at",
        "status",
        "issue_type",
        "priority",
        "reassigned_flag",
        "reopen_count"
    ],
    sep="_"
)

interactions_df.head()


,interaction_id,timestamp,channel,interaction_type,advocate_id_at_time,notes,case_id,patient_id,advocate_id,opened_at,closed_at,status,issue_type,priority,reassigned_flag,reopen_count
0,I-1001-1,2025-11-01T09:20:00,phone,initial_contact,A-001,Patient called about an unexpected charge on r...,C-1001,P-0501,A-001,2025-11-01T09:15:00,2025-11-05T14:40:00,closed,billing,high,0,0
1,I-1001-2,2025-11-02T10:05:00,portal,follow_up,A-001,Advocate requested itemized statement from bil...,C-1001,P-0501,A-001,2025-11-01T09:15:00,2025-11-05T14:40:00,closed,billing,high,0,0
2,I-1001-3,2025-11-04T16:30:00,phone,update,A-001,Explained corrected charges and confirmed part...,C-1001,P-0501,A-001,2025-11-01T09:15:00,2025-11-05T14:40:00,closed,billing,high,0,0
3,I-1001-4,2025-11-05T14:30:00,phone,closure,A-001,Patient confirmed understanding of final bill ...,C-1001,P-0501,A-001,2025-11-01T09:15:00,2025-11-05T14:40:00,closed,billing,high,0,0
4,I-1002-1,2025-10-01T09:20:00,phone,initial_contact,A-002,Patient called about an unexpected charge on r...,C-1002,P-0502,A-002,2025-10-01T09:15:00,2025-10-05T14:40:00,closed,billing,high,0,0


In [15]:
cases_df.to_csv("/kaggle/working/cases.csv", index=False)
interactions_df.to_csv("/kaggle/working/interactions.csv", index=False)


In [19]:
import random
from datetime import datetime, timedelta

issue_types = ["billing", "scheduling", "insurance_coverage", "referral", "medical_records", "prescription", "provider_change"]
priorities = ["low", "medium", "high", "urgent"]
statuses = ["open", "in_progress", "closed"]
channels = ["phone", "portal", "email", "sms"]
interaction_types = ["initial_contact", "follow_up", "update", "escalation", "closure"]

def random_dt(start_dt, end_dt):
    delta = end_dt - start_dt
    seconds = random.randint(0, int(delta.total_seconds()))
    return start_dt + timedelta(seconds=seconds)

base_open = datetime(2025, 11, 1, 8, 0, 0)

auto_cases = []
start_index = len(cases_raw) + 1  # continue after your manual cases

for i in range(start_index, 41):  # up to around C-1030
    case_id = f"C-{1000 + i}"
    patient_id = f"P-{random.randint(1000, 2999)}"
    advocate_id = f"A-{str(random.randint(1, 15)).zfill(3)}"

    opened_at = base_open + timedelta(hours=random.randint(0, 240))
    is_closed = random.random() < 0.7
    closed_at = random_dt(opened_at, opened_at + timedelta(days=7)) if is_closed else None
    status = "closed" if is_closed else random.choice(["open", "in_progress"])
    issue_type = random.choice(issue_types)
    priority = random.choice(priorities)
    reassigned_flag = random.choice([0, 1])
    reopen_count = random.choice([0, 0, 1])  # mostly zero

    interactions = []
    n_interactions = random.randint(2, 6)
    current_advocate = advocate_id

    for j in range(1, n_interactions + 1):
        ts = random_dt(opened_at, (closed_at or (opened_at + timedelta(days=5))))
        if reassigned_flag and j == 2:
            current_advocate = f"A-{str(random.randint(1, 15)).zfill(3)}"

        itype = "initial_contact" if j == 1 else random.choice(interaction_types[1:])
        interaction = {
            "interaction_id": f"I-{1000 + i}-{j}",
            "timestamp": ts.isoformat(timespec="minutes"),
            "channel": random.choice(channels),
            "interaction_type": itype,
            "advocate_id_at_time": current_advocate,
            "notes": f"Synthetic {itype.replace('_', ' ')} for {issue_type} case."
        }
        interactions.append(interaction)

    case = {
        "case_id": case_id,
        "patient_id": patient_id,
        "advocate_id": advocate_id,
        "opened_at": opened_at.isoformat(timespec="minutes"),
        "closed_at": closed_at.isoformat(timespec="minutes") if closed_at else None,
        "status": status,
        "issue_type": issue_type,
        "priority": priority,
        "reassigned_flag": reassigned_flag,
        "reopen_count": reopen_count,
        "interactions": sorted(interactions, key=lambda x: x["timestamp"])
    }

    auto_cases.append(case)

len(auto_cases)


31

In [20]:
all_cases = cases_raw + auto_cases
len(all_cases)


40

In [21]:
cases_df = pd.json_normalize(
    all_cases,
    sep="_",
    meta=[
        "case_id",
        "patient_id",
        "advocate_id",
        "opened_at",
        "closed_at",
        "status",
        "issue_type",
        "priority",
        "reassigned_flag",
        "reopen_count"
    ]
)
if "interactions" in cases_df.columns:
    cases_df = cases_df.drop(columns=["interactions"])

interactions_df = pd.json_normalize(
    all_cases,
    record_path="interactions",
    meta=[
        "case_id",
        "patient_id",
        "advocate_id",
        "opened_at",
        "closed_at",
        "status",
        "issue_type",
        "priority",
        "reassigned_flag",
        "reopen_count"
    ],
    sep="_"
)

cases_df.shape, interactions_df.shape


((40, 10), (157, 16))

In [22]:
import json

with open("/kaggle/working/advocacy_cases_full.json", "w") as f:
    json.dump(all_cases, f, indent=2)

cases_df.to_csv("/kaggle/working/cases_full.csv", index=False)
interactions_df.to_csv("/kaggle/working/interactions_full.csv", index=False)


USE

In [24]:
import pandas as pd

cases_df = pd.read_csv("/kaggle/working/cases_full.csv")
interactions_df = pd.read_csv("/kaggle/working/interactions_full.csv")

cases_df.head(), interactions_df.head()


(  case_id patient_id advocate_id            opened_at            closed_at  \
 0  C-1001     P-0501       A-001  2025-11-01T09:15:00  2025-11-05T14:40:00   
 1  C-1002     P-0502       A-002  2025-10-01T09:15:00  2025-10-05T14:40:00   
 2  C-1003     P-0503       A-003  2025-10-02T09:15:00  2025-10-06T14:40:00   
 3  C-1004     P-0504       A-004  2025-10-01T09:15:00  2025-10-05T14:40:00   
 4  C-1005     P-0505       A-005  2025-10-07T09:15:00  2025-10-09T14:40:00   
 
    status issue_type priority  reassigned_flag  reopen_count  
 0  closed    billing     high                0             0  
 1  closed    billing     high                0             0  
 2  closed    billing     high                0             0  
 3  closed    billing     high                0             0  
 4  closed    billing   medium                0             0  ,
   interaction_id            timestamp channel interaction_type  \
 0       I-1001-1  2025-11-01T09:20:00   phone  initial_contact   
 1   

In [26]:
cases_df["opened_at"] = pd.to_datetime(cases_df["opened_at"], format="ISO8601")
cases_df["closed_at"] = pd.to_datetime(cases_df["closed_at"], format="ISO8601")
interactions_df["timestamp"] = pd.to_datetime(interactions_df["timestamp"], format="ISO8601")


In [27]:
first_interaction = (
    interactions_df
    .sort_values("timestamp")
    .groupby("case_id")["timestamp"]
    .min()
    .rename("first_response_at")
    .reset_index()
)
first_interaction.head()


,case_id,first_response_at
0,C-1001,2025-11-01 09:20:00
1,C-1002,2025-10-01 09:20:00
2,C-1003,2025-10-02 09:20:00
3,C-1004,2025-10-01 09:20:00
4,C-1005,2025-10-07 09:20:00


In [28]:
case_metrics = cases_df.merge(first_interaction, on="case_id", how="left")

case_metrics["time_to_first_response_hours"] = (
    (case_metrics["first_response_at"] - case_metrics["opened_at"])
    .dt.total_seconds() / 3600
)
case_metrics.head()


,case_id,patient_id,advocate_id,opened_at,closed_at,status,issue_type,priority,reassigned_flag,reopen_count,first_response_at,time_to_first_response_hours
0,C-1001,P-0501,A-001,2025-11-01 09:15:00,2025-11-05 14:40:00,closed,billing,high,0,0,2025-11-01 09:20:00,0.083333
1,C-1002,P-0502,A-002,2025-10-01 09:15:00,2025-10-05 14:40:00,closed,billing,high,0,0,2025-10-01 09:20:00,0.083333
2,C-1003,P-0503,A-003,2025-10-02 09:15:00,2025-10-06 14:40:00,closed,billing,high,0,0,2025-10-02 09:20:00,0.083333
3,C-1004,P-0504,A-004,2025-10-01 09:15:00,2025-10-05 14:40:00,closed,billing,high,0,0,2025-10-01 09:20:00,0.083333
4,C-1005,P-0505,A-005,2025-10-07 09:15:00,2025-10-09 14:40:00,closed,billing,medium,0,0,2025-10-07 09:20:00,0.083333


In [29]:
# Resolution time (hours) – will be NaN for open cases
case_metrics["resolution_time_hours"] = (
    (case_metrics["closed_at"] - case_metrics["opened_at"])
    .dt.total_seconds() / 3600
)

# Interactions per case
interaction_counts = (
    interactions_df
    .groupby("case_id")["interaction_id"]
    .count()
    .rename("interaction_count")
    .reset_index()
)

case_metrics = case_metrics.merge(interaction_counts, on="case_id", how="left")

case_metrics[["case_id", "time_to_first_response_hours", "resolution_time_hours", "interaction_count"]].head()


,case_id,time_to_first_response_hours,resolution_time_hours,interaction_count
0,C-1001,0.083333,101.416667,4
1,C-1002,0.083333,101.416667,7
2,C-1003,0.083333,101.416667,8
3,C-1004,0.083333,101.416667,6
4,C-1005,0.083333,53.416667,4


In [33]:
# Define a simple 'open' flag from status
case_metrics["is_open"] = case_metrics["status"].isin(["open", "in_progress"]).astype(int)

backlog_by_advocate = (
    case_metrics
    .groupby("advocate_id")["is_open"]
    .sum()
    .rename("open_case_count")
    .reset_index()
)

backlog_by_advocate.head()


,advocate_id,open_case_count
0,A-001,0
1,A-002,0
2,A-003,0
3,A-004,0
4,A-005,0


In [34]:
# Save enriched case metrics
case_metrics.to_csv("/kaggle/working/case_metrics.csv", index=False)

# Save backlog by advocate
backlog_by_advocate.to_csv("/kaggle/working/backlog_by_advocate.csv", index=False)
